In [1]:
# standard modules
import importlib

# PyTorch modules
import torch

# import data + model modules
import ukbb_data
import ukbb_ica_models
# in case of changes 
importlib.reload(ukbb_data)
importlib.reload(ukbb_ica_models)

# import custom functions
import utils
# in case of changes
importlib.reload(utils)

# check: is GPU available?
torch.cuda.is_available()

In [2]:
# prepare data paths
ukbb_dir = '/ritter/share/data/UKBB/ukb_data/'

# define logging path
path = '../tracking/ICA25/KernelSize1DCNN/'

In [ ]:
utils.make_reproducible()

In [7]:
! gpustat

cuda02                      Tue Jun 13 15:54:37 2023  470.182.03
[0] NVIDIA GeForce RTX 3090 | 29'C,   0 % |  4204 / 24268 MB | tomasz(2873M) tjade(1323M) gdm(4M)
[1] NVIDIA GeForce RTX 3090 | 49'C,   2 % |  1913 / 24268 MB | laurar(1905M) gdm(4M)
[2] NVIDIA GeForce RTX 3090 | 55'C,  62 % | 11352 / 24268 MB | tjade(2417M) tjade(8927M) gdm(4M)
[3] NVIDIA GeForce RTX 3090 | 65'C,  82 % | 17271 / 24268 MB | tjade(17263M) gdm(4M)


# Testing different kernel sizes for the ICA25 simple 1D CNN

In [ ]:
def train_kernelsize(log_path, data_path, kernel_size, device):
    """
    Fuction for using the same model, training and testing set-up for testing
    different kernel sizes.
    Outputs test score and a plot visualising the training progression.
    Input:
        log_path: path to where logs, checkpoints and data info should be saved
        data_path: path to location where data is saved (expectations see utils.DataModule)
        kernel_size: kernel size to set on all layers, as int
        device: which GPU to run on
    Output:
        trainer: trained model
        datamodule: PyTorch Lightning UKBB DataModule
    """
    # initialise model
    simple_CNN = ukbb_ica_models.simple1DCNN(kernel_size=kernel_size)

    # initialise logger
    logger = utils.logger_init(save_dir=log_path+str(kernel_size)+'/')

    # set callbacks
    early_stopping = utils.earlystopping_init()

    checkpoint = utils.checkpoint_init(save_dir=log_path+str(kernel_size)+'/')

    # initialise trainer
    trainer = utils.trainer_init(device=device,
                                logger=logger,
                                callbacks=[early_stopping, checkpoint])

    # initialise DataModule
    datamodule = ukbb_data.UKBBDataModule(data_path)

    # train model
    trainer.fit(simple_CNN, datamodule=datamodule)
    print('Training complete.')

    # save info on which data was used + what the train/val/test split was
    utils.save_data_info(path=log_path+str(kernel_size)+'/', datamodule=datamodule)
    
    return trainer, datamodule

def test_kernelsize(trainer, datamodule, kernel_size):
    """
    Fuction for using the same model and testing set-up for testing
    different kernel sizes.
    Outputs test score and a plot visualising the training progression.
    Input:
        trainer: the trained model
        datamodule: PyTorch Lightning DataModule instance
        kernel_size: kernel size to set on all layers, as int
    """
    print(f'\nTesting model with kernel size {kernel_size}...')
    
    # test model
    trainer.test(ckpt_path='best', datamodule=datamodule)
    
    # visualise training
    print(f'\nVisualise training of model with kernel size {kernel_size}')    
    metrics = utils.get_current_metrics(trainer, show=True)
    utils.plot_training(data=metrics, title=f'Training visualisation of the ICA25 1D-CNN with kernel size {kernel_size}')

## kernel size = 3

In [ ]:
kernelsize_3, kernelsize_3_data = train_kernelsize(path, ukbb_dir, 3, 1)

In [ ]:
test_kernelsize(kernelsize_3, kernelsize_3_data, 3)

## kernel size = 5

In [ ]:
kernelsize_5, kernelsize_5_data = train_kernelsize(path, ukbb_dir, 5, 1)

In [ ]:
test_kernelsize(kernelsize_5, kernelsize_5_data, 5)

## kernel size = 7

In [ ]:
kernelsize_7, kernelsize_7_data = train_kernelsize(path, ukbb_dir, 7, 1)

In [ ]:
test_kernelsize(kernelsize_7, kernelsize_7_data, 7)

## kernel size = 10

In [ ]:
kernelsize_10, kernelsize_10_data = train_kernelsize(path, ukbb_dir, 10, 1)

In [ ]:
test_kernelsize(kernelsize_10, kernelsize_10_data, 10)

## kernel size = 15

In [ ]:
kernelsize_15, kernelsize_15_data = train_kernelsize(path, ukbb_dir, 15, 1)

In [ ]:
test_kernelsize(kernelsize_15, kernelsize_15_data, 15)